<a href="https://colab.research.google.com/github/MaximeCornaton/CPE_IA-DeepLearning/blob/master/DEPOs_TP1_TP2/TP2/TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy scikit-learn matplotlib

# TP2

## Exercice 2.1

In [2]:
import numpy as np

class Perceptron:

    def __init__(self, num_inputs, activation_function, learning_rate, num_epochs):
        self.__num_inputs = num_inputs
        # Créer un tableau de longueur num_inputs contenant les poids (entre 0 et 1) associés aux entrées
        self.__weights = np.random.rand(num_inputs)
        # Choix d'un biais random (entre 0 et 1)
        self.__bias = np.random.rand()
        self.__activation_function = activation_function
        self.__learning_rate = learning_rate
        self.__num_epochs = num_epochs

    def train(self, X, y):
        for epoch in range(self.__num_epochs):
            for i in range(len(X)):
                input_data = X[i]
                prediction = self.predict(input_data)
                error = y[i] - prediction
                self.__weights += self.__learning_rate * error * input_data
                self.__bias += self.__learning_rate * error
        
    def predict(self, X):
        linear_combination = self.__bias
        for i in range(len(X)):
            linear_combination += X[i] * self.__weights[i]
        return self.activate(linear_combination)

    def activate(self, x):
        if self.__activation_function == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        elif self.__activation_function == 'tanh':
            return np.tanh(x)
        elif self.__activation_function == 'relu':
            return np.maximum(0, x)
        elif self.__activation_function == 'gaussian':
            return np.exp(-x ** 2)
        else:
            # Renvoyer la fonction d'activation identité par défaut
            return x




if __name__ == "__main__":
    # Exemple de test avec une fonction SIGMOID
    # L'objectif est de démontrer la capacité du perceptron à apprendre la logique OR
    X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    y = np.array([0, 1, 1, 1])

    perceptron = Perceptron(num_inputs=2, activation_function='sigmoid', learning_rate=0.5, num_epochs=1000)
    perceptron.train(X, y)

    # Test des prédictions
    test_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    for data in test_data:
        prediction = perceptron.predict(data)
        print(f"Prévision pour {data}: {prediction}")

Prévision pour [0 0]: 0.010114057576340141
Prévision pour [0 1]: 0.9959831278740803
Prévision pour [1 0]: 0.9959675063637465
Prévision pour [1 1]: 0.9999998331582493


In [3]:
import numpy as np

class MultiLayerPerceptron:
    def __init__(self, num_inputs, num_hidden, num_outputs, activation='sigmoid', epochs=1000, learning_rate=0.1):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.activation = activation
        
        # Initialisation des poids pour les couches cachées et de sortie
        self.hidden_weights = np.random.rand(self.num_inputs, self.num_hidden)
        self.output_weights = np.random.rand(self.num_hidden, self.num_outputs)

        # Choix de la fonction d'activation
        if activation == 'sigmoid':
            self.activation_function = self._sigmoid
            self.activation_derivative = self._sigmoid_derivative
        elif activation == 'tanh':
            self.activation_function = np.tanh
            self.activation_derivative = lambda x: 1 - np.tanh(x)**2
        elif activation == 'relu':
            self.activation_function = self._relu
            self.activation_derivative = self._relu_derivative
        else:
            raise ValueError("Fonction d'activation non prise en charge")

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _sigmoid_derivative(self, x):
        return x * (1 - x)

    def _relu(self, x):
        return np.maximum(0, x)

    def _relu_derivative(self, x):
        return np.where(x <= 0, 0, 1)

    def train(self, X, y):
        for epoch in range(self.epochs):
            # Forward propagation
            hidden_layer_input = np.dot(X, self.hidden_weights)
            hidden_layer_output = self.activation_function(hidden_layer_input)

            output_layer_input = np.dot(hidden_layer_output, self.output_weights)
            output_layer_output = self.activation_function(output_layer_input)

            # Calcul de l'erreur
            output_error = y - output_layer_output
            
            # Calcul du gradient pour la couche de sortie
            output_delta = output_error * self.activation_derivative(output_layer_output)

            # Calcul de l'erreur pour la couche cachée
            hidden_error = output_delta.dot(self.output_weights.T)
            
            # Calcul du gradient pour la couche cachée
            hidden_delta = hidden_error * self.activation_derivative(hidden_layer_output)

            # Mise à jour des poids
            self.output_weights += hidden_layer_output.T.dot(output_delta) * self.learning_rate
            self.hidden_weights += X.T.dot(hidden_delta) * self.learning_rate

    def predict(self, X):
        hidden_layer_input = np.dot(X, self.hidden_weights)
        hidden_layer_output = self.activation_function(hidden_layer_input)

        output_layer_input = np.dot(hidden_layer_output, self.output_weights)
        output_layer_output = self.activation_function(output_layer_input)

        return output_layer_output

# Exemple d'utilisation avec une fonction XOR
if __name__ == "__main__":
    # Données d'entrée et de sortie pour la fonction XOR
    X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    y = np.array([[0], [1], [1], [0]])

    # Création et entraînement du modèle
    model = MultiLayerPerceptron(num_inputs=2, num_hidden=4, num_outputs=1, activation='sigmoid', epochs=10000, learning_rate=0.1)
    model.train(X, y)

    # Prédictions
    predictions = model.predict(X)
    print("Prédictions finales:")
    for i in range(len(predictions)):
        print(f"Données d'entrée: {X[i]}, Prédiction: {predictions[i]}")


Prédictions finales:
Données d'entrée: [0 0], Prédiction: [0.11073436]
Données d'entrée: [0 1], Prédiction: [0.90991055]
Données d'entrée: [1 0], Prédiction: [0.91726987]
Données d'entrée: [1 1], Prédiction: [0.07019086]


## Exercice 2.2

## Exercice 2.3

## Exercice 2.4